In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import allel
import pandas as pd

This notebook is one attempt to create a method to identify a minimal set of predictors (SNPs) which fully describe a non-ordinal categorical response (haplotypes) for a given region of the genome. My idea is to perform a (possibly multinomial) logistic regression, using regularization with an L1 penalization parameter, which should (hopefully) shrink the coefficients for many SNPs to 0.

There are many different methods for finding "tag SNPs" which can capture the variance explained by multiple surrounding SNPs. This problem is non-trivial and potentially NP-hard. See https://doi.org/10.1109/CSB.2005.22 for one example and some background on different methods. Note that much of this work was performed around the time of the first human HapMap project (ca. 2005), and hence there are many links to software on websites which no longer exist.

Many tag SNP identification algorithms are designed to find sets of tag SNPs across chromosomes/genomes, without any phenotypic data. I think that my particular use-case here is actually somewhat simpler, since I:

1. Have access to a "phenotype" (haplotype groups determined by a simple distance matrix)
2. Only need to focus on one region, obviating the need to find haploblock boundaries

At least that's what I tell myself - maybe this won't work at all. See https://chrisalbon.com/machine_learning/logistic_regression/logistic_regression_with_l1_regularization/ and http://alimanfoo.github.io/2017/06/14/read-vcf.html for some sources on SciKit functionality that have influenced my thinking, for better or worse.

In [4]:
## User-defined constants
vcf_file = '/mnt/DATAPART2/brian/repos/manuscripts/manu_2020_Yr/input_data/geno/GAWN_KM_Yr_postimp_filt.vcf.gz'
reg = '1A:526943215-598847646'

haps_file = '/mnt/DATAPART2/brian/repos/manuscripts/manu_2020_Yr/temp/combined_graph_out_w_priors/haplotype_groupings.csv'

In [5]:
## Read in the VCF file region
vcf = allel.read_vcf(vcf_file, region = reg)
#dir(allel)

In [11]:
help(vcf.keys)
#gt = vcf['calldata/GT']
#gt.head()

Help on built-in function keys:

keys(...) method of builtins.dict instance
    D.keys() -> a set-like object providing a view on D's keys

